# Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from context import rmlib
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot( robotConfig )

#Imports 
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.assembly_trees.asm_tree_Basic import augment_RM

augment_RM( robot )

%store -r

####### LIBS #######################################################################################################################################

# Import Pytrees
import py_trees
from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector
from py_trees.decorators import FailureIsSuccess , SuccessIsFailure

# import 
from rmlib.rmtools.assembly_trees.asm_tree_Basic import *

from rmlib.rmtools.assembly_trees.asm_tree_FT_based import ( 
    Bias_Wrist , Spiral_Insert , Tilt_Insert , Spin_Press , Move_to_Contact , SpiralStep
)
    
from rmlib.rmtools.assembly_trees.asm_tree_drill import *

from rmlib.rmtools.assembly_trees.asm_tree_logic_flow import Run_to_X_Failures_DECO

import math
from pmath import *

from utils import is_matx_list
from math import radians

print( "Libs loaded!" )

from task_parameters import taskParams , _TB_side , _M8_TB_height , _M6_TB_height , _M4_TB_height

_DRILL_CONTACT_PRESS = 11.0

import pickle , traceback

def save_names_to_PKL( pklPath , nameStringList ):
    pklStruct = {}
    print( vars() )
    for name in nameStringList:
        pklStruct[ name ] = globals()[ name ]
        
    with open( pklPath , "wb" ) as f:
        try:
            print( "Pickling" , pklPath , end = '... ' )
            pickle.dump( pklStruct , f )
            print( "Success!" )
        except Exception as er:
            print( "FAILURE! :" , er )
            traceback.print_exc()
    print( "### Completed Writing" , pklPath , "###" )
    
def load_names_from_PKL_to_global( pklPath , nameStringList ):
    n = 0
    with open( pklPath , "rb" ) as f:
        try:
            print( "Unpickling" , pklPath , end = '... ' )
            pklStruct = pickle.load( f )
            print( "Got: " , type( pklStruct ) , "with" , len( pklStruct ) , "elements" )
            globDict = globals()
            for name in nameStringList:
                try:
                    globals()[ name ] = pklStruct[ name ]
                    n += 1
                except:
                    print( "Could not load '" + str( name ) + "'! Continue ..." )
        except Exception as er:
            print( "FAILURE! :" , er )
            traceback.print_exc()
            
    print( "### Read" , n , "vars from" , pklPath , "into global dict ###" )

load_names_from_PKL_to_global( 
    pklPath = "taughtPoses.pkl" , 
    nameStringList = [  
        'M8_GraspPose' , 'M6_GraspPose' , 'M4_GraspPose' , 'GearShaft1_Setdown' , 'GearShaft2_Setdown' , 
        'BNC_Ma_Setdown' , 'KeySm_Setdown' , 'KeyLg_Setdown' , 'ShaftSm_Setdown' , 'ShaftLg_Setdown' , 
        'KeyLg_Grasp' , 'ShaftLg_Grasp' , 'BNC_Fe_Grasp' , 'ShaftSm_Grasp' , 
        'GearLg_Grasp' , 'GearLg_Grasp' , 'Ether_setdown' , 
        'M8_1_mag' , 'M8_2_mag' , 'M8_3_mag' , 'M8_4_mag' , 'M8_5_mag' , 'M8_6_mag' ,
        'M6_1_mag' , 'M6_2_mag' , 'M6_3_mag' , 'M6_4_mag' , 'M6_5_mag' , 'M6_6_mag' ,
        'M4_1_mag' , 'M4_2_mag' , 'M4_3_mag' , 'M4_4_mag' , 'M4_5_mag' , 'M4_6_mag' ,
        'releasePose' , 'SAFE_POSE' , 'midDrillPose' ,
        '_LRG_GEAR_DEX' , '_SML_GEAR_DEX', 'aboveTableDrill'
    ]
)

load_names_from_PKL_to_global( 
    pklPath = "dct.pkl" , 
    nameStringList = [ "dct" ]
)


## BIAS TEST ##
if 0:
    for i in range(40):
        print('##', i+1, '##')
        before = robot.ft.get_wrist_force()
        bgn    = time.time()
        robot.ft.bias_wrist_force()
        print( time.time() - bgn , "[s] bias time" )
        after  = robot.ft.get_wrist_force()
        print( after )
        print( "Diff:" , np.linalg.norm( np.subtract( after, before ) ) , '\n\n' )

# Arm Movements

In [ ]:
"""
### Move to Safe Pose ###
Basic movements are defined in:
rmlib/rmlib/rmtools/assembly_trees/asm_tree_Basic.py

All of the basic behaviors and trees inherit `py_trees` classes.
""" 

moveNode = Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.125 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB


## Run Behavior
`run_BT_until_done` is a function that ticks the node object provided as an argument at regular intervals.  It will report the final status of the root node and print and ASCII tree representation.   
The default args are likely adequate for your purpose, but it can be found at  
rmlib/rmlib/rmtools/assembly_trees/asm_tree_Basic.py


run_BT_until_done( moveNode )



## Pose Math, Move to Contact, and Tree Building

In [ ]:
# Calc a pose 10cm below the `SAFE_POSE`
lowPose = translate_pose( 
    SAFE_POSE , 
    [ 0.0 , 0.0 , -0.100 ] , # Translation vector
    dir_pose = 'origin' # { 'origin': lab frame, 'self': relative the argument pose }
)
# Location:  rmlib/rmlib/rmtools/pmath.py

The following cell builds a sequence that makes a relative move, then stores the final pose under a global dictionary key for later retrieval.  
For most 'procedure-like' trees, a Sequence with Memory is used.  
The memory option prevents completed actions from being ticked again until the entire `Sequence` has completed.  
Behaviors will be executed in the order added.

In [ ]:
pushTree = Sequence( memory = 1 ) 

poseKey = "TouchdownPose" # String key to store pose that will be used by another tree

# 1. Move to a pose conveniently close to the table
pushTree.add_child(
    Move_Arm( pose = lowPose , mode = 'l',  speed = 0.125 , accel = 0.35 , ctrl = robot ) 
)

# 2. Close the fingers in prep for a probing action
pushTree.add_child(
    Set_Fingers( 
        openState = 0.0 , # Dist b/n fingers in [m], Anything larger than 0.095 interpreted as fully open
        ctrl = robot , # rmlib ref
        waitSec = 0.25 # OpenCM does not block execution, so pause to let fingers physically close
    )
)

"""
### Move Until Reaction Force Encountered ###
Force-based actions are defined in:
rmlib/rmlib/rmtools/assembly_trees/asm_tree_FT_based.py
""" 

# 3. Approach the table at a **gentle speed** and stop when a small reaction force is observed 
pushTree.add_child(
    Move_to_Contact( 
        Fmag      = 3.0 , # Reaction force that triggers stop, Forces below 1.5N aren't reliably detected
        relMove   = [0.0, 0.0, -1.0] , # Specify a move the goes *beyond* where you expect to end up
        biasWrist = 1 , # Bias wrist before beginning motion (Sorry, currently this takes 1-8 seconds)
        mode      = 'l' , 
        speed     = 0.040 , accel = 0.04 , # **gentle speed**
        pull      = 0 , # For pulling actions, reaction force tugs on hand rather than pushes
        ctrl      = robot
    ) # NOTE: This behav. may be used for absolute pose if an argument for `pose` is provided
)

# 4. Remember where we landed
pushTree.add_child(
    Store_Current_Pose( 
        keyString = poseKey , # Blackboard dict key
        ctrl = robot , 
        setOnce = True # Set this flag to prevent the value from being modified once stored
    )
)

# 5. Relative move back up, no condition
pushTree.add_child(
    Move_Arm_Relative( 
        translation = [ -0.035, -0.035, 0.070 ] , # Trans. Vec
        rotation    = [0.0,0.0,0.0] , # Euler angles
        mode = 'l' , speed = 0.125 , accel = 0.35 , 
        stop_cond = None , # A custon stop condition can be passed here if desired
        frame = 'origin' ,
        cond_success = 1 , # Flag true if we are using "stop_cond()==1" to mean *success*, otherwise the reverse
        bias_wrist = 0 , # If `stop_cond` is force based, this is a good idea
        ctrl = robot
    ) # NOTE: If no stop_cond is provided, the this behavior returns true only if the full motion has completed
)

# 6. Now let's recover the pose that we stored before
pushTree.add_child(
    Move_Arm( BB_key = poseKey , # Blackboard dict key passed to `BB_key` instead of a `pose` arg
              mode = 'l',  speed = 0.063 , accel = 0.17 , ctrl = robot ) 
)

# 7. It's a good idea to return to a known safe pose between activities
pushTree.add_child(
    Move_Arm( pose  = SAFE_POSE , ctrl = robot )
)

# 8. Open the fingers
pushTree.add_child(
    Set_Fingers( openState = 1.0 , ctrl = robot , waitSec = 0.0 )
)

print()

# Utils & Setup

In [ ]:
robot.hand.deactivate_motors()

In [ ]:
robot.arm.get_tcp_pose()

In [ ]:
robot.hand.set_finger_torque(0.3)

In [ ]:
robot.hand.set_finger_width(0.0)

In [ ]:
robot.arm.move_speed( midDrillPose , 'l' , 0.125 , 0.35 , 0 , 'dummy' , False )

In [ ]:
# midDrillPose = robot.arm.get_tcp_pose() #rm.get_tcp_pose()
# %store midDrillPose

In [ ]:
robot.arm.move_speed( aboveTableDrill , 'l' , 0.125 , 0.35 , 0 , 'dummy' , False )

# Run this after teaching any poses

In [ ]:
import pickle , traceback

save_names_to_PKL( 
    pklPath = "taughtPoses.pkl" , 
    nameStringList = [  
        'startPose' ,
        'M8_GraspPose' , 'M6_GraspPose' , 'M4_GraspPose' , 'GearShaft1_Setdown' , 'GearShaft2_Setdown' , 
        'BNC_Ma_Setdown' , 'KeySm_Setdown' , 'KeyLg_Setdown' , 'ShaftSm_Setdown' , 'ShaftLg_Setdown' , 
        'KeyLg_Grasp' , 'ShaftLg_Grasp' , 'BNC_Fe_Grasp' , 'ShaftSm_Grasp' , 
        'GearLg_Grasp' , 'GearLg_Grasp' , 'Ether_setdown' , 
        'M8_1_mag' , 'M8_2_mag' , 'M8_3_mag' , 'M8_4_mag' , 'M8_5_mag' , 'M8_6_mag' ,
        'M6_1_mag' , 'M6_2_mag' , 'M6_3_mag' , 'M6_4_mag' , 'M6_5_mag' , 'M6_6_mag' ,
        'M4_1_mag' , 'M4_2_mag' , 'M4_3_mag' , 'M4_4_mag' , 'M4_5_mag' , 'M4_6_mag' ,
        'releasePose' , 'SAFE_POSE' , 'midDrillPose' , 
        '_LRG_GEAR_DEX' , '_SML_GEAR_DEX', 'aboveTableDrill'
    ]
)